# Import thư viện

In [ ]:
import requests
import pandas as pd
from pathlib import Path


# Tạo nơi lưu trữ dữ liệu

In [ ]:
OUT = Path(r"C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw")
OUT.mkdir(parents=True, exist_ok=True)

# Crawl dữ liệu trên worldbank bằng API

In [ ]:
# ===============================
# 🌍 TẢI DỮ LIỆU TỪ WORLD BANK API
# ===============================

def crawl_worldbank_raw(indicator: str, country: str = "VN", per_page: int = 1000) -> pd.DataFrame:
    """
    Hàm này tải dữ liệu chỉ số kinh tế (indicator) từ World Bank API theo quốc gia.
    Trả về DataFrame chứa dữ liệu thô.
    
    Ví dụ:
        indicator = "NY.GDP.MKTP.CD"   # GDP (current US$)
        country   = "VN"               # Vietnam
    """

    # URL cơ bản cho API World Bank (v2)
    base = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"
    page = 1                # Bắt đầu từ trang 1
    all_rows = []           # Danh sách lưu toàn bộ các bản ghi

    # =============================
    # 🔁 VÒNG LẶP LẤY DỮ LIỆU TỪNG TRANG
    # =============================
    while True:
        # Các tham số truy vấn (query params)
        params = {
            "format": "json",      # Định dạng trả về là JSON
            "per_page": per_page,  # Số lượng bản ghi mỗi trang (mặc định 1000)
            "page": page           # Trang hiện tại
        }

        # Gửi yêu cầu HTTP GET đến API World Bank
        r = requests.get(base, params=params, timeout=30)
        r.raise_for_status()  # Nếu có lỗi HTTP (404, 500, ...) → ném ngoại lệ

        # Parse phản hồi JSON
        payload = r.json()

        # Dữ liệu World Bank có dạng [metadata, data]
        # metadata: thông tin chung (tổng số trang, tổng số bản ghi, ...)
        # data: danh sách các bản ghi cụ thể
        if not isinstance(payload, list) or len(payload) < 2 or payload[1] is None:
            break  # Nếu không hợp lệ → dừng

        meta, data = payload[0], payload[1]
        if not data:
            break  # Nếu không còn dữ liệu → thoát

        # =============================
        # 🧩 CHUYỂN TỪNG ITEM TRONG JSON → DICTIONARY PYTHON
        # =============================
        for item in data:
            row = {
                "indicator_id":    (item.get("indicator") or {}).get("id"),     # Mã chỉ số (VD: NY.GDP.MKTP.CD)
                "indicator_value": (item.get("indicator") or {}).get("value"),  # Tên chỉ số (VD: GDP (current US$))
                "country_id":      (item.get("country") or {}).get("id"),       # Mã quốc gia (VD: VN)
                "country_value":   (item.get("country") or {}).get("value"),    # Tên quốc gia (VD: Vietnam)
                "countryiso3code": item.get("countryiso3code"),                 # ISO3 code (VD: VNM)
                "date":            item.get("date"),                            # Năm dữ liệu (VD: 2023)
                "value":           item.get("value"),                           # Giá trị của chỉ số (VD: GDP = 409 billion)
                "unit":            item.get("unit"),                            # Đơn vị đo (thường là None)
                "obs_status":      item.get("obs_status"),                      # Trạng thái quan sát (thường None)
                "decimal":         item.get("decimal"),                         # Số chữ số thập phân
            }
            all_rows.append(row)  # Thêm vào danh sách tổng

        # =============================
        # ⏹️ KIỂM TRA XEM CÒN TRANG NÀO KHÔNG
        # =============================
        total_pages = meta.get("pages", 1)  # Lấy tổng số trang từ metadata
        if page >= total_pages:             # Nếu đã tới trang cuối → thoát
            break
        page += 1                           # Ngược lại → sang trang tiếp theo

    # =============================
    # 🧾 CHUYỂN THÀNH DATAFRAME PANDAS
    # =============================
    df = pd.DataFrame(all_rows, columns=[
        "indicator_id", "indicator_value",
        "country_id", "country_value", "countryiso3code",
        "date", "value", "unit", "obs_status", "decimal"
    ])

    # Trả về DataFrame hoàn chỉnh
    return df


# ===============================
# 💾 HÀM LƯU FILE CSV SAU KHI CRAWL
# ===============================

def save_worldbank_raw(indicator: str, country: str = "VN", out_dir: Path = None) -> Path:
    """
    Gọi hàm crawl_worldbank_raw() để lấy dữ liệu, sau đó lưu ra file CSV.
    Trả về đường dẫn file CSV đã lưu.
    """

    # 1️⃣ Gọi hàm tải dữ liệu
    df = crawl_worldbank_raw(indicator, country)

    # 2️⃣ Nếu không truyền thư mục lưu → dùng thư mục hiện tại
    if out_dir is None:
        out_dir = Path(".")

    # 3️⃣ Tạo thư mục nếu chưa tồn tại
    out_dir.mkdir(parents=True, exist_ok=True)

    # 4️⃣ Tạo đường dẫn file xuất
    out_path = out_dir / f"{indicator}_{country}_RAW.csv"

    # 5️⃣ Lưu DataFrame thành CSV (encoding UTF-8-sig để không lỗi font tiếng Việt)
    df.to_csv(out_path, index=False, encoding="utf-8-sig")

    # 6️⃣ In thông báo kết quả ra console
    print(f"✅ Lưu {len(df)} dòng -> {out_path}")

    # 7️⃣


Gửi request api trên worldbank và crawl dữ liệu về sau đó chuyển thành dataframe rồi lưu vào file 

Crawl những dữ liệu như sau:

-GDP (USD hiện tại)

-FDI inflows (Dòng vốn đầu tư trực tiếp nước ngoài / USD )

-GDP growth (Tăng trưởng của GDP / % )

# Lưu dữ liệu ra file đã chỉ định


In [ ]:
# GDP (current USD)
save_worldbank_raw("NY.GDP.MKTP.CD", "VN", OUT)

In [ ]:

# FDI inflows (USD)
save_worldbank_raw("BX.KLT.DINV.CD.WD", "VN", OUT)

In [ ]:
# GDP growth (%)
save_worldbank_raw("NY.GDP.MKTP.KD.ZG", "VN", OUT)
